# 3.1 Introduction to Sockets

A **socket** in Python refers to the endpoint of a communication channel most generally. That is, if we have a connection between two devices, we have a socket associated with each device. We can have many different types of sockets depending on the channel of communication, e.g internet or bluetooth socket. In this notebook, we will cover the basics of TCP (Transmission Control Protocol) sockets. More on [TCP](https://www.youtube.com/watch?v=CRdL1PcherM&t=238s) and [networking models](https://www.youtube.com/watch?v=3kfO61Mensg) in general.

## TCP Internet Socket

Let us create a TCP internet socket,

In [1]:
import socket

# Creating socket object for a TCP internet connection,
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

Remember that _socket.AF_INET_ specifies that we have an **internet connection** while _socket.SOCK_STREAM_ tells the socket to use TCP as the protocol.

### Basic Client-Server Connection

Let us now consider creating a server and then connecting a cilent to it via sockets. The machine running the server should execute the code contained in _server.py_. We must specify a port through which the server will communicate with the client. 

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# server.py

"""This code will be ran by the server."""

import socket

SERVER_PORT = 56781

# Getting the server's host IP,
host_ip = socket.gethostbyname(socket.gethostname())

# Creating the socket object,
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Opening the socket,
server_socket.bind((host_ip, SERVER_PORT)) # <--- IP and port must be provided.
server_socket.listen(5) # <-- Socket has a limit of five connections.

while True:

    # Accepting any client,
    client, client_ip = server_socket.accept()

    # Sending message to client,
    connected_string = "[Server]: Connection established!"
    print("[Server (you)]: Connection established via {} at {}".format(client, client_ip))
    client.send(connected_string.encode()) # <-- We need to encode messages before sending them.

    # Closing connection,
    client.close()

Once the connection with the client has been made, an encoded message in the form of a **byte stream** is sent. The client machine should execute the code contained in _client.py_.

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# client.py

"""This code will be ran by the cilent."""

import socket

SERVER_PORT = 56781

# In this case the client is the same device as the server,
server_ip = socket.gethostbyname(socket.gethostname())

# Creating the client socket,
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connecting the cilent to the sever by connecting the sockets,
client_socket.connect((server_ip, SERVER_PORT))

# Accepting and decoding the message sent from the server,
message = client_socket.recv(1024).decode()
print(message)

# closing the client socket,
client_socket.close()

The client socket is created and we attempt to connect to the server socket. When connected, we accepted and decode the message send by the server. Our last step is to close the client socket.

### One Way TCP Chat (Server-Client)

Now let us write more advanced code which allows two-way communication between the client and the server. Code for the server machine,

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# server.py

"""This code will be ran by the server."""

import socket
import threading

# GLOBAL VARIABLES,
SERVER_PORT = 56781

class Server():

    def __init__(self, port):

        # The port the server will communicate to clients by,
        self.port = port
        self.client_connected = False
        self.client_port = None

        # Getting the server's host IP,
        self.host_ip = socket.gethostbyname(socket.gethostname())

        # Creating the socket object,
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Opening the socket,
        self.server_socket.bind((self.host_ip, self.port)) # <--- IP and port must be provided.
        self.server_socket.listen(5) # <-- Socket has a limit of five connections.

    def accept_client(self):

        while self.client_connected == False:

            # Accepting any client,
            print("[Server (You)]: Waiting for client connection...")
            self.client, self.client_address = self.server_socket.accept()

            # Sending message to client,
            connected_string = "[Server]: Connection established."
            print("[Server (You)]: Connection established via {} at {}".format(self.client, self.client_address))
            self.client.send(connected_string.encode()) # <-- We need to encode messages before sending them.

            # Breaking out of loop,
            self.client_connected = True

    def send_message(self, string):

        try:
            message_string = "[Server]: " + string
            self.client.send(message_string.encode())
        except Exception as error:
            print("[Server (You)]: " + str(error))

    def receive_message(self):

        while True:
            # Accepting and decoding the message sent from the client,
            message_string = self.client.recv(1024).decode()
            print(message_string)

if __name__ == "__main__":

    banner = """
 ____                           
/ ___|  ___ _ ____   _____ _ __ 
\___ \ / _ \ '__\ \ / / _ \ '__|
 ___) |  __/ |   \ V /  __/ |   
|____/ \___|_|    \_/ \___|_|
"""

    print(banner)
    print("--------------------------------------------------------")

    server = Server(SERVER_PORT)
    server.accept_client()

    incoming_thread = threading.Thread(target = server.receive_message)
    incoming_thread.start()

    while True:
        string = input()
        server.send_message(string)

For the client machine,

In [ ]:
#!/usr/bin/env python
# coding: utf-8
# client.py

"""This code will be ran by the cilent."""

import socket
import threading

# GLOBAL VARIABLES,
SERVER_ADDRESS = (socket.gethostbyname(socket.gethostname()), 56781)
"""^In this case, the client is the same device as the server. If not, change the host name."""

class Client():

    def __init__(self, server_address):

        self.server_address = server_address

        # Creating the client socket,
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        # Connecting the cilent to the sever by connecting the sockets,
        print("[Client (You)]: Attempting connection to server...")
        self.client_socket.connect(server_address)

        # Accepting and decoding the message sent from the server,
        connected_string = self.client_socket.recv(1024).decode()
        print(connected_string)

    def receive_message(self):

        while True:

            # Accepting and decoding the message sent from the server,
            message_string = self.client_socket.recv(1024).decode()
            print(message_string)

    def send_message(self, string):

        try:
            # Sending message to server,
            message_string = "[Client]: " + string
            self.client_socket.send(message_string.encode())
        except Exception as error:
            print("[Client (You)]: " + str(error))

if __name__ == "__main__":

    banner = """
  ____ _ _            _   
 / ___| (_) ___ _ __ | |_ 
| |   | | |/ _ \ '_ \| __|
| |___| | |  __/ | | | |_ 
 \____|_|_|\___|_| |_|\__|
 """

    print(banner)
    print("--------------------------------------------------------")

    client = Client(SERVER_ADDRESS)
    incoming_thread = threading.Thread(target = client.receive_message)
    incoming_thread.start()

    while True:
        string = input()
        client.send_message(string)

More work is required regarding the way in which printing the messages are handled. Simultaneous typing results in improper printing.